### 배송지에 따른 사기성

In [ ]:
#shipping address에서 주만을 추출하는 코드
data['State'] = data['Shipping Address'].apply(lambda x: x.split(',')[-1].strip().split(' ')[0])

In [ ]:
#배송지에 따른 사기성을 평균, 총합계, 사기성이 일어난 값을 추출
state_fraud = data.groupby('State')['Is Fraudulent'].agg(['mean', 'count', 'sum']).reset_index()
#mean값을 퍼센트로 보기 위해 100 곱하기
state_fraud['mean'] = state_fraud['mean'] * 100
#mean으로 sort한 값 추출
state_fraud_sorted = state_fraud.sort_values(by='mean', ascending=False)

### Shipping Address와 Billing Address에서 주소가 같은 경우와 다를 경우의 사기성 차이

In [ ]:
#결제할 때 작성한 주소와 제품이 배송된 주소가 동일한 값 추출
data['Address Match'] = data['Shipping Address'] == data['Billing Address']

address_match_fraud= data.groupby('Address Match')['Is Fraudulent'].agg(['mean', 'count','sum']).reset_index()
address_match_fraud['mean'] = address_match_fraud['mean'] * 100

In [ ]:
#시각화
plt.figure(figsize=(8, 6))
bar_plot = sns.barplot(data=address_match_fraud, x='Address Match', y='mean', palette='viridis')
plt.title('Fraud Rate Based on Address Match')
plt.xlabel('Address Match')
plt.ylabel('Fraud Rate (%)')
plt.grid(True, axis='y')

#그래프에 값 작성
for p in bar_plot.patches:
    bar_plot.annotate(format(p.get_height(), '.2f'), 
                      (p.get_x() + p.get_width() / 2., p.get_height()),
                      ha = 'center', va = 'center', 
                      xytext = (0, 9), 
                      textcoords = 'offset points')

plt.tight_layout()
plt.show()


### 특정 주의 결제 수단에서 사기성 거래가 더 많이 발생

In [ ]:
payState_fraud = data.groupby(['State', 'Payment Method'])['Is Fraudulent'].agg(['mean', 'count','sum'])
payState_fraud ['mean'] = payState_fraud ['mean'] * 100
payState_fraud_sorted = payState_fraud.sort_values(by='mean', ascending=False)

In [ ]:
# 트랜젝션이 거의 없는 부분을 삭제
filtered_fraud_rate = payState_fraud[payState_fraud['count'] >= 30]

#인덱스 재설정
filtered_fraud_rate = filtered_fraud_rate.reset_index()

#시각화
plt.figure(figsize=(14, 8))
line_plot = sns.lineplot(data=filtered_fraud_rate, x='State', y='mean', hue='Payment Method', marker='o')
plt.title('Fraud Rate by State and Payment Method')
plt.xlabel('State')
plt.ylabel('Fraud Rate')
plt.xticks(rotation=45) 
plt.legend(title='Payment Method', bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.grid(True)
plt.tight_layout() 
plt.show()

### 가설 - 가장 사기성을 많이 띈 USNV 주에서 가장 높게 측정된 상품 카테고리, 장치유형, 거래시간, 지불방법을 조사

In [ ]:
#state가 USNV일 때(USNV가 가장 높은 사기성을 띔)
usnv_data = data[data['State'] == 'USNV']

# 상품 카테고리, 장치유형, 거래시간, 지불방법을 조사
category_freq = usnv_data['Product Category'].value_counts().idxmax()
device_freq = usnv_data['Device Used'].value_counts().idxmax()
time_freq = usnv_data['Transaction Hour'].value_counts().idxmax()
payment_freq = usnv_data['Payment Method'].value_counts().idxmax()

fig, axs = plt.subplots(2, 2, figsize=(14, 12))

#상품 카테고리
sns.countplot(data=usnv_data, x='Product Category', ax=axs[0, 0], palette='viridis')
axs[0, 0].set_title('Product Category Distribution in USNV')
axs[0, 0].set_xlabel('Product Category')
axs[0, 0].set_ylabel('Count')
axs[0, 0].tick_params(axis='x', rotation=45)

#장치유형
sns.countplot(data=usnv_data, x='Device Used', ax=axs[0, 1], palette='viridis')
axs[0, 1].set_title('Device Used Distribution in USNV')
axs[0, 1].set_xlabel('Device Used')
axs[0, 1].set_ylabel('Count')

#거래시간
sns.countplot(data=usnv_data, x='Transaction Hour', ax=axs[1, 0], palette='viridis')
axs[1, 0].set_title('Transaction Hour Distribution in USNV')
axs[1, 0].set_xlabel('Transaction Hour')
axs[1, 0].set_ylabel('Count')

#지불방법
sns.countplot(data=usnv_data, x='Payment Method', ax=axs[1, 1], palette='viridis')
axs[1, 1].set_title('Payment Method Distribution in USNV')
axs[1, 1].set_xlabel('Payment Method')
axs[1, 1].set_ylabel('Count')
axs[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()